# AI Agents — Foundations Guide

This notebook introduces AI Agents comprehensively: concepts, architectures, usage guidance, and runnable scripts.

## Table of Contents (80 Topics)

### Part 1 — Agent Fundamentals (6)
1. What are AI Agents?
2. Agent Architectures (Reactive, Deliberative, Hybrid)
3. Agent Components (Perception, Planning, Action, Memory)
4. Agent Types (Goal/Utility/Learning)
5. Environments (deterministic/stochastic, episodic/sequential)
6. Lifecycle & Event Loops

### Part 2 — Architectures & Patterns (6)
7. Reactive Agents (Behavior-based)
8. Deliberative Agents (Planning)
9. Hybrid Architectures (Sense-Plan-Act + Reactive)
10. Modular Agent Design
11. Agent-Oriented Programming
12. Communication Patterns

### Part 3 — Reasoning & Planning (6)
13. Symbolic Planning (PDDL, STRIPS)
14. Heuristic Search (A*, Best-First)
15. Task & Motion Planning
16. Probabilistic Planning (POMDPs)
17. Hierarchical Task Networks (HTN)
18. Real-time Planning & Replanning

### Part 4 — Learning & Adaptation (6)
19. Reinforcement Learning Agents
20. Imitation Learning & Behavioral Cloning
21. Offline vs Online RL
22. Meta-Learning for Agents
23. Continual Learning
24. Transfer & Multi-task Learning

### Part 5 — Language-Capable Agents (6)
25. LLM Agents (Planner/Executor)
26. ReAct & Thought-Action Loops
27. Tool-Enabled Agents (APIs, DBs)
28. RAG-Enhanced Agents
29. Conversational Dialog Management
30. Safety & Alignment

### Part 6 — Tools & Integrations (6)
31. Tool Design (idempotency, side-effects)
32. Authorization & Sandboxing
33. Orchestration & Workflows
34. Observability & Tool Logging
35. External Knowledge (APIs, KB, DBs)
36. Testing & Sandboxes

### Part 7 — Memory & State (6)
37. Short-term vs Long-term Memory
38. Vector Memory & Embeddings
39. Retrieval Strategies (recency+relevance)
40. Summarization & Condensation
41. Privacy & Retention Policies
42. Grounding with Citations

### Part 8 — Multi-Agent Systems (6)
43. Coordination & Negotiation
44. Emergent Behavior
45. Protocols & ACLs
46. Distributed Planning & Consensus
47. Market/Auction Mechanisms
48. Safety in Multi-Agent Contexts

### Part 9 — Evaluation & Metrics (6)
49. Success Metrics & Rewards
50. Latency & Cost Metrics
51. Groundedness & Hallucination Rate
52. Human Preference & UX
53. Robustness to Shift
54. Interpretability & Auditability

### Part 10 — Safety & Governance (6)
55. Validator & Simulator Layers
56. Ethics (bias, fairness)
57. Governance & Access Control
58. Red-Team Testing
59. Fail-safe & Degradation
60. User Consent & Control

### Part 11 — Deployment & Scaling (6)
61. Edge vs Cloud Agents
62. Autoscaling
63. Caching & Partial Results
64. Monitoring & Alerting
65. Cost Optimization
66. Versioning & Rollbacks

### Part 12 — Advanced Patterns (6)
67. Agent-of-Agents / Meta-Agents
68. Self-Reflective Agents
69. Curriculum Learning
70. Human-in-the-Loop Escalation
71. Hybrid Symbolic-LLM Agents
72. Simulation Agents

### Part 13 — Frameworks & Tooling (8)
73. LangChain Agent Patterns
74. AutoGen / Colang / TaskMatrix
75. Microsoft/Anthropic/OpenAI SDKs
76. Agent Benchmarks & Eval Toolkits
77. Debugging (traces, replays)
78. Security & Policy Enforcement
79. Reference Projects & Templates
80. Future Directions & Challenges

In [ ]:
# Create a blank notebook using nbformat
import nbformat as nbf
nb = nbf.v4.new_notebook()
nb.cells = [
    nbf.v4.new_markdown_cell("# New Notebook"),
    nbf.v4.new_code_cell("print('Hello from new notebook')")
]
with open("new_notebook.ipynb", "w", encoding="utf-8") as f:
    nbf.write(nb, f)

# Why: reproducible project scaffolding, CI/CD notebook generation.
# When: initializing analysis pipelines, templating notebooks per dataset.

## Topic 1 — What are AI Agents?

### Definition
An AI agent is a software entity that perceives inputs, maintains state, plans actions, and executes tools to achieve goals, optionally learning over time.

### Components
- Perception: APIs, sensors, events, text
- Memory: short-term context + long-term knowledge
- Reasoning/Planning: rules, search, RL, LLMs
- Tools/Actuators: API clients, DB queries, file I/O
- Executor: deterministic side-effectful actions

### Why Use AI Agents
- Automate multi-step workflows (search → analyze → act)
- Maintain context & memory across interactions
- Safely access external systems via tools
- Improve reliability with retries, validation, and escalation

### When NOT to Use
- Single-turn Q&A without tool use
- Ultra low-latency systems (<100ms)
- Fully deterministic pipelines (traditional code simpler)

### Risks & Mitigations
- Hallucinations → retrieval + validators
- Unsafe tools → sandbox + allow-list + prompts
- Privacy → retention policies + redaction
- Cost → caching, rate limits, cheaper models

### Real-World Examples
- Customer support assistants (FAQ search + ticketing)
- Research assistants (papers search + summarization)
- DevOps bots (deploy, monitor, rollback)
- Personal assistants (calendar, email, tasks)

In [ ]:
# Topic 1 Code — Minimal Agent (Planner + Tools + Memory)
from dataclasses import dataclass
from typing import Callable, Dict, Any, List
import shlex

@dataclass
class Tool:
    name: str
    func: Callable[..., Any]
    description: str = ""

class MiniAgent:
    def __init__(self):
        self.tools: Dict[str, Tool] = {}
        self.memory: List[str] = []

    def add_tool(self, tool: Tool):
        self.tools[tool.name] = tool

    def plan(self, text: str) -> Dict[str, str]:
        t = text.lower()
        if any(k in t for k in ['calculate', '+', '-', '*', '/']):
            return {'action': 'use_tool', 'tool': 'calculator', 'input': text}
        if any(k in t for k in ['search', 'policy', 'faq', 'find']):
            return {'action': 'use_tool', 'tool': 'kb', 'input': text}
        if 'remember' in t or 'note' in t:
            return {'action': 'remember', 'input': text}
        return {'action': 'use_tool', 'tool': 'kb', 'input': text}

    def execute(self, plan: Dict[str, str]) -> str:
        if plan['action'] == 'remember':
            self.memory.append(plan['input'])
            return '✓ remembered'
        if plan['action'] == 'use_tool':
            tool = self.tools.get(plan['tool'])
            if not tool:
                return f"no such tool: {plan['tool']}"
            try:
                return str(tool.func(plan['input']))
            except Exception as e:
                return f"tool error: {e}"
        return 'unknown action'


def calculator(expr: str):
    tokens = shlex.split(expr)
    s = ''.join(tokens)
    safe = set('0123456789+-*/().%')
    filtered = ''.join(ch for ch in s if ch in safe)
    return eval(filtered, {'__builtins__': {}})


def kb(query: str):
    data = {
        'vacation days': '15 days PTO; accrues 1.25 days/month',
        'remote work': 'Up to 3 days/week with approval',
        'health insurance': 'Employee pays 20%; starts after 30 days',
    }
    q = query.lower()
    for k, v in data.items():
        if k in q or any(w in q for w in k.split()):
            return v
    return 'no match in KB'

agent = MiniAgent()
agent.add_tool(Tool('calculator', calculator, 'Evaluate arithmetic'))
agent.add_tool(Tool('kb', kb, 'Simulated policy KB'))

print(agent.execute(agent.plan('Calculate 15 * (12 + 3.5)')))
print(agent.execute(agent.plan('What are my vacation days?')))
print(agent.execute(agent.plan('Please remember I prefer dark mode')))

## Topic 2 — Agent Architectures

### Reactive (Reflex) Agents
- Map percepts → actions via rules; ultra-fast
- Use when latency is critical (robotics, alerts)

### Deliberative (Planning) Agents
- Build internal model and search for action sequences
- Use for multi-step tasks, constraints, longer-horizon goals

### Hybrid Agents
- Combine reactive safety with deliberative planning
- Recommended for assistants needing both responsiveness and reasoning

### LLM Planner / Deterministic Executor
- LLM decides next step; executor runs tools safely
- Pros: flexible reasoning, auditable tool calls; Cons: latency, cost

### Decision Guide
- Real-time control → Reactive
- Complex multi-step workflows → Deliberative
- Assistants with tools → Hybrid/LLM Planner

### Considerations
- Error handling & rollback
- Authorization & sandboxing
- Observability & tracing
- Memory windows & retrieval

In [ ]:
# Topic 2 Code — Reactive, Deliberative, Hybrid, LLM-based demos

class ReactiveAgent:
    def __init__(self):
        self.rules = []
    def add_rule(self, condition, action):
        self.rules.append((condition, action))
    def act(self, percept: str) -> str:
        for condition, action in self.rules:
            if condition(percept):
                return action(percept)
        return "No rule matched"

reactive = ReactiveAgent()
reactive.add_rule(lambda p: 'danger' in p.lower(), lambda p: '⚠️ ALERT!')
reactive.add_rule(lambda p: 'temperature' in p.lower(), lambda p: '🌡️ Monitoring temperature')
print('Reactive:', reactive.act('Danger detected'))

class DeliberativeAgent:
    def act_deliberatively(self, goal: str) -> str:
        steps = ['search data', 'analyze results', 'report summary']
        return f"Plan for '{goal}': " + ' → '.join(steps)

print('Deliberative:', DeliberativeAgent().act_deliberatively('market analysis'))

class HybridAgent:
    def __init__(self):
        self.reactive_rules = []
        self.goals = {}
    def add_reactive_rule(self, cond, act):
        self.reactive_rules.append((cond, act))
    def add_goal(self, name, steps):
        self.goals[name] = steps
    def act(self, percept: str) -> str:
        for cond, act in self.reactive_rules:
            if cond(percept):
                return f"REACTIVE: {act(percept)}"
        for name, steps in self.goals.items():
            if name in percept.lower():
                return f"PLANNING: {name} → " + ' → '.join(steps)
        return 'No match'

hybrid = HybridAgent()
hybrid.add_reactive_rule(lambda p: 'emergency' in p.lower(), lambda p: 'Activate protocol')
hybrid.add_goal('book flight', ['search', 'check price', 'confirm'])
print('Hybrid:', hybrid.act('book flight to NYC'))

class LLMBasedAgent:
    def __init__(self):
        self.tools = {}
    def register_tool(self, name, func):
        self.tools[name] = func
    def plan(self, query: str):
        q = query.lower()
        if 'calculate' in q:
            return {'tool': 'calculator', 'input': query}
        if 'weather' in q:
            return {'tool': 'weather', 'input': 'current'}
        return {'tool': None, 'input': None}
    def execute(self, plan):
        if plan['tool'] is None:
            return 'Direct answer'
        tool = self.tools.get(plan['tool'])
        if not tool:
            return 'Tool missing'
        return tool(plan['input'])

llm = LLMBasedAgent()
llm.register_tool('weather', lambda _: '🌤️ Sunny 72°F')
llm.register_tool('calculator', lambda q: eval(q.replace('calculate', '').strip()))
plan = llm.plan('Calculate 25 * 4')
print('LLM-based:', llm.execute(plan))